In [7]:
import os
import dotenv

from azure.ai.ml import MLClient
from azure.identity import AzureCliCredential

dotenv.load_dotenv()
subscription = os.getenv(f"subscription_id")
resource_group = os.getenv(f"resource_group")
workspace = os.getenv(f"workspace_name")

ml_client = MLClient(
    AzureCliCredential(), 
    subscription, 
    resource_group, 
    workspace,
)

In [36]:
from azure.ai.ml import command, Input, Output
from azure.ai.ml.constants import AssetTypes, InputOutputModes


# Azure Machine Learning workspace details:
subscription = os.getenv(f"subscription_id")
resource_group = os.getenv(f"resource_group")
workspace = os.getenv(f"workspace_name")
datastore_name = 'workspaceblobstore'
path_on_datastore = 'stroppel_videos/2023-03-28/'
output_path_on_datastore = 'stroppel_tracking_output/2023-03-28/'

# long-form Datastore uri format:
uri = f'azureml://subscriptions/{subscription}/resourcegroups/{resource_group}/workspaces/{workspace}/datastores/{datastore_name}/paths/'
uri_input = f'{uri}{path_on_datastore}'
uri_output = f'{uri}{output_path_on_datastore}'

inputs = {
    "input_data": Input(type="uri_folder",
              path=uri_input,
              mode=InputOutputModes.RO_MOUNT,
              )
}
outputs = {
    "output_data": Output(type="uri_folder",
                path=uri_output,
                mode=InputOutputModes.RW_MOUNT,
                # name="stroppel_tracking_output"
                )
}

arguments = [
    "--tracking_config", "kalman_tracking_settings.yaml",
    "--dataset", "${{inputs.input_data}}",
    "--output", "${{outputs.output_data}}",
]

job = command(
    code=".",
    command="python run_on_azure/kalman_tracking_azure.py " + " ".join(arguments),
    environment="fishy_environment@latest",
    compute="Standard-A1-v2",
    experiment_name="batch_tracking",
    inputs=inputs,
    outputs=outputs,
)

In [37]:
import shutil
# copy library files to job source directory temporarily
pth = './algorithm/'
shutil.copytree('../algorithm/', pth)

returned_job = ml_client.create_or_update(job)

shutil.rmtree(pth)

Uploading run_on_azure (3.58 MBs): 100%|##########| 3583798/3583798 [00:00<00:00, 4397059.88it/s]


